# ESTIMATIONS AND RESULTS

# Reset everything and redo data cleaning

In [120]:
%reset -f

In [121]:
import pandas as pd 
import numpy as np 
import numpy.linalg as la
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from scipy.stats import norm
from sklearn.preprocessing import PolynomialFeatures

# Indlæser data og databehandler på samme måde som før

In [122]:
dat = pd.read_csv('growth.csv')
lbldf = pd.read_csv('labels.csv', index_col='variable')
lbl_all = lbldf.label.to_dict() # as a dictionary
print(f'The data contains {dat.shape[0]} rows (countries) and {dat.shape[1]} columns (variables).')

The data contains 214 rows (countries) and 85 columns (variables).


# Defines variable groups

In [123]:
# all available variables
vv_institutions = ['marketref', 'dem', 'demCGV', 'demBMR', 'demreg'] 
vv_geography = [
        'tropicar','distr', 'distcr', 'distc','suitavg','temp', 'suitgini', 'elevavg', 'elevstd',
        'kgatr', 'precip', 'area', 'abslat', 'cenlong', 'area_ar', 'rough','landlock', 
        'africa',  'asia', 'oceania', 'americas' # 'europe' is the reference
]
vv_geneticdiversity = ['pdiv', 'pdiv_aa', 'pdivhmi', 'pdivhmi_aa']
vv_historical = ['pd1000', 'pd1500', 'pop1000', 'pop1500', 'ln_yst'] # these are often missing: ['pd1', 'pop1']
vv_religion = ['pprotest', 'pcatholic', 'pmuslim']
vv_danger = ['yellow', 'malfal',  'uvdamage'] # Diseases
vv_resources = ['oilres', 'goldm', 'iron', 'silv', 'zinc']
vv_educ = ['ls_bl', 'lh_bl'] # secondary, tertiary: we exclude 'lp_bl' (primary) to avoid rank failure 

vv_all = {'institutions': vv_institutions, 
          'geography': vv_geography, 
          'geneticdiversity': vv_geneticdiversity,
          'historical': vv_historical,
          'religion': vv_religion,
          'danger':vv_danger, 
          'resources':vv_resources
         }
list_of_lists = vv_all.values()
vv_all['all'] = [v for sublist in list_of_lists for v in sublist]

# Variables we use
# Institutions, geography, religion, danger (health), resources, education, pop growth

# Code job for Lucas
# Use variables choosen
# Graph Lasso with different penalty levels
# Lasso with CV, BRT and BCCH at difference penalty levels like in week 4 exercise
# Use BRT and BCCH optimal penalty level and use dobbel lasso and post dobbelt lasso.



# Removes countries with missing growth and initial gdp

In [124]:
from datetime import datetime

# ---- configuration ----
id_col = 'code'
cols_to_check = ['gdp_growth', 'gdp_pc_initial']

# ---- identify rows to remove / keep ----
missing_mask = dat[cols_to_check].isnull().any(axis=1)
removed = dat.loc[missing_mask].copy()
kept = dat.loc[~missing_mask].copy()
print(f"Rows with missing {cols_to_check} ({removed.shape[0]} rows) have been removed.")
print(f"Remaining data has {kept.shape[0]} rows.")


Rows with missing ['gdp_growth', 'gdp_pc_initial'] (112 rows) have been removed.
Remaining data has 102 rows.


# Defines variables

In [125]:
# Keep only interesting variables
# Combining all variables we want to use
vs = []
vs.extend(vv_all['institutions'])  # Institutional variables
vs.extend(vv_all['geography'])     # Geographic variables
vs.extend(vv_all['religion'])      # Religious variables
vs.extend(vv_all['danger'])        # Health/disease variables
vs.extend(vv_all['resources'])     # Natural resource variables
vs.extend(vv_educ)                 # Education variables
vs.extend(['pop_growth'])          # Population growth
vs.extend(['lgdp_initial'])        # Log gdp initial
vs.extend(['pdiv'])              # Genetic diversity

# Add our main variables of interest
main_var = ['gdp_pc_initial']
all_vars = main_var + vs + ['gdp_growth']

# Only remove countries with missing values in our chosen variables
complete_cases = dat[all_vars].notnull().all(axis=1)
kept = dat[complete_cases].copy()


# Prepare data for Lasso
y = kept['gdp_growth'].values.reshape((-1,1)) * 100.0  # Convert to percentage
Z_basic = kept[vs].values       # All variables except the dependent
d = kept['gdp_pc_initial'].values # Our beta variable

Xnames = list(kept[['gdp_pc_initial'] + vs].columns)

# Print dimensions of our matrices
print(f'Z_basic matrix shape: {Z_basic.shape}')
print(f'y vector shape: {y.shape}')

# Display number of regressors
#print("The number of regressors in Z is {}".format(Z.shape[1]))

# Verify no missing values remain in our variables of interest
print("\nMissing values after filtering:")
print(kept[all_vars].isnull().sum())


Z_basic matrix shape: (72, 42)
y vector shape: (72, 1)

Missing values after filtering:
gdp_pc_initial    0
marketref         0
dem               0
demCGV            0
demBMR            0
demreg            0
tropicar          0
distr             0
distcr            0
distc             0
suitavg           0
temp              0
suitgini          0
elevavg           0
elevstd           0
kgatr             0
precip            0
area              0
abslat            0
cenlong           0
area_ar           0
rough             0
landlock          0
africa            0
asia              0
oceania           0
americas          0
pprotest          0
pcatholic         0
pmuslim           0
yellow            0
malfal            0
uvdamage          0
oilres            0
goldm             0
iron              0
silv              0
zinc              0
ls_bl             0
lh_bl             0
pop_growth        0
lgdp_initial      0
pdiv              0
gdp_growth        0
dtype: int64


# Removes 3 variables with multicolirations

In [126]:
# Drop unwanted variables from the kept DataFrame and update related arrays/lists
cols_to_drop = ['uvdamage', 'demCGV', 'demBMR']
# We drop these, as they are very correlated with other variables, leading to multicollinearity issues.

# Drop from kept if present
for c in cols_to_drop:
    if c in kept.columns:
        kept.drop(columns=c, inplace=True)

# Remove from vs (controls) if present
vs = [v for v in vs if v not in cols_to_drop]

# Rebuild X, d, Z and Xnames consistent with the dropped variables
Z_basic = kept[vs].values
d = kept['gdp_pc_initial'].values

# Defines X
X = np.column_stack((d, Z_basic))
Xnames = list(kept[['gdp_pc_initial'] + vs].columns)

# Also makes polynomial for Z
Z = PolynomialFeatures(2, include_bias=False).fit_transform(Z_basic)
print("Shape of Z with degree k:", Z.shape)
X_pol = np.column_stack((d, Z))
Xnames_pol = ['gdp_pc_initial'] + [f'Z{i}' for i in range(Z.shape[1])]

# Find N
N = X.shape[0]
N_pol = X_pol.shape[0]

# Update variable name lists if they exist in the namespace
for name in ('variable_names', 'var_names', 'var_names_reduced', 'vars_no_const'):
    if name in globals():
        globals()[name] = [v for v in globals()[name] if v not in cols_to_drop]

print("Dropped columns:", [c for c in cols_to_drop if c not in kept.columns])
print("X shape:", X.shape)
print("d shape:", d.shape)

print("First X names:", Xnames[:10])
print("First X_pol names (polynomial):", Xnames_pol[:10])

Shape of Z with degree k: (72, 819)
Dropped columns: ['uvdamage', 'demCGV', 'demBMR']
X shape: (72, 40)
d shape: (72,)
First X names: ['gdp_pc_initial', 'marketref', 'dem', 'demreg', 'tropicar', 'distr', 'distcr', 'distc', 'suitavg', 'temp']
First X_pol names (polynomial): ['gdp_pc_initial', 'Z0', 'Z1', 'Z2', 'Z3', 'Z4', 'Z5', 'Z6', 'Z7', 'Z8']


# Makes Standarization

In [127]:
# Create a function for standardizing
def standardize(X):

    X_stan = (X - np.mean(X, axis=0))/np.std(X, axis=0, ddof=1)
    return X_stan

# Standardize data
X_stan = standardize(X)
X_pol_stan = standardize(X_pol)
Z_stan = standardize(Z_basic)
Z_stan_pol = standardize(Z)
d_stan = standardize(d)

C:\Users\lucas\AppData\Local\Temp\ipykernel_13076\294159085.py:4: RuntimeWarning: invalid value encountered in divide
  X_stan = (X - np.mean(X, axis=0))/np.std(X, axis=0, ddof=1)


In [128]:
# Tests
print(f'Mean y = {y.mean(): 5.2f}% growth per year')
print(f'Mean d = {d.mean(): 5.2f} log initial GDP per capita')


Mean y =  1.58% growth per year
Mean d =  6677.53 log initial GDP per capita


# 1: OLS

In [129]:
# Add a constant to X
xx = np.column_stack((np.ones(N),X))

# Reshape y
yy = np.array(y).reshape(-1,1)

# Calculate OLS estimate
coefs_OLS = la.inv(xx.T@xx)@xx.T@yy
alpha_OLS = coefs_OLS[1][0]

# Calculate residuals
res_OLS = yy - xx@coefs_OLS

# Display alpha
print("alpha_OLS = ",alpha_OLS.round(6))

alpha_OLS =  3.5e-05


# Double Post Lasso no polynomial

Define BRT penalty

In [130]:
# Make a function that calculates BRT. Hint: You implemented a version of this last week
def BRT(X_tilde,y):
    (N,p) = X_tilde.shape
    sigma = np.std(y, ddof=1)
    c=1.1
    alpha=0.05

    penalty_BRT= (sigma*c)/np.sqrt(N)*norm.ppf(1-alpha/(2*p))

    return penalty_BRT

In [131]:
# Calculate BRT
penalty_BRTyx = BRT(X_stan, y)
print("lambda_BRT =",penalty_BRTyx.round(2))

lambda_BRT = 0.6


BCCH

In [132]:
def BCCH(X,y):

    n,p = X.shape
    c = 1.1; alpha = 0.05

    coef_pilot = Lasso(alpha).fit(X,y).coef_
    coef_intercept = Lasso(alpha).fit(X,y).intercept_
    pred = (coef_intercept + X@coef_pilot)

    res = y - pred

    resXscale = (np.max((X.T ** 2.0) @ (res ** 2.0) / n)) ** 0.5
    lambda_bcch = c*norm.ppf(1.0-alpha/(2.0*p))*resXscale/np.sqrt(n)    
    penalty_BCCH = lambda_bcch

    return penalty_BCCH  

In [133]:
# Calculate BCCH
penalty_BCCH  = BCCH(X_stan, y)
print("lambda_BCCH =",penalty_BCCH.round(2))

lambda_BCCH = 2.42


In [134]:
# Run Lasso 
fit_BRTyx = Lasso(penalty_BRTyx, max_iter=10000).fit(X_stan, y)
coefs=fit_BRTyx.coef_

# Calculate residuals
resyx = y-fit_BRTyx.predict(X_stan)

# Calculate Y - Z@gamma (epsilon + alpha*d)
# Hint: You only need the variables given to you in this cell, in addition
# to a standardized data set you made previoously.
resyxz = resyx + d_stan*coefs[0]

# Display first coefficient
print("First coefficient =",coefs[0].round(5))

First coefficient = -0.0


Calculates BRT for treatment

In [138]:
# Calculate BRT
penalty_BRTdz = BRT(Z_stan, d)

In [ ]:
# Run Lasso
fit_BRTdz = Lasso(penalty_BRTdz, max_iter=10000).fit(Z_stan, d)
coefs=fit_BRTdz.coef_

# Calculate residuals
resdz=d-fit_BRTdz.predict(Z_stan)

# Display first coefficient
print("First coefficient =",coefs[0].round(2))

[ 7084.18058853 26120.62107297 19573.86716246 19807.72539172
   795.15641066  1399.51518635  4704.31756123   601.91744846
  4552.25558836   228.51534634  1940.40147513   930.99026227
  2879.25125707  3732.06314988 19681.32338568 30541.92957522
  1699.03077242  2708.46404566  2487.73368819   730.42301601
 13571.9176367  18267.45802309 19984.58614096  7205.8702031
 17874.81341695  1077.13956926   743.21646832 13279.29421718
  1897.29751549  2948.84762406  1309.80325758   772.12967809
   396.01169193 12730.4260683   6965.44390263 17671.33697522
   598.45742617  1793.44928787   711.83955457  1019.25200138
  5524.27238582   360.78973466   170.71763681  2004.32812096
   293.15515174  1915.87155007   817.3483333   2162.74450459
 24519.60923614 32244.64637681   284.41456431   469.43771537
  3434.72613789  3352.03480769  1305.15163407  8759.96732015
  1726.89912219   355.80836447 22133.91201239  1207.32022755
   476.5572005   2554.18438196 26656.77933081   621.80626228
   929.09371124  1253.383

Estimates alpha

In [140]:
# Calculate alpha
num = resdz@resyxz
denom = resdz@d
alpha_PDL = num/denom

# Display alpha
print("alpha_PDL = ",alpha_PDL.round(2))

alpha_PDL =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
